<a href="https://colab.research.google.com/github/alfianpr/perhitungan-volume-objek-simetri/blob/main/case_1/Shopee.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [110]:
#Load semua library yang dibutuhkan
from selenium import webdriver
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service

#menyiapkan dataset yang nanti akan berisi data hasil scraping
dataset = pd.DataFrame(columns=['SKU', 'Harga', 'Jumlah_Penjualan', 'Halaman'])

#Setting webdriver versi chrome
s = Service('/usr/local/bin/chromedriver.exe')
driver = webdriver.Chrome(service=s)

#Fungsi looping untuk mengekstrak informasi yang dibutuhkan dari setiap halaman
for pg in range(0,1): #Halaman yang mau di scrape
  page_num = str(pg)
  brand = "rockbros" #Nama brand yang mau dicari informasinya
  shopee_url = 'https://shopee.co.id/search?keyword={}&page='.format(brand)+page_num
  driver.get(shopee_url)
  
  #Informasi-informasi yang dibutuhkan
  nama = driver.find_elements(By.XPATH, '//div[@class="_10Wbs- _5SSWfi UjjMrh"]')
  harga = driver.find_elements(By.XPATH, '//span[@class="_1d9_77"]')
  penjualan = driver.find_elements(By.XPATH, '//div[@class="_2VIlt8"]')

  #Membuat list data nama, harga dan penjumlahan untuk menampung data hasil scraping
  nama_list = []
  for p in range(len(nama)):
    try : nama_list.append(nama[p].text)
    except : nama_list.append("0")

  harga_list = []
  for x in range(len(harga)):
    try : harga_list.append(harga[x].text.replace('.', ''))
    except : harga_list.append("0")

  penjualan_list = []
  for z in range(len(penjualan)):
    try : penjualan_list.append(penjualan[z].text.split()[0].replace('RB','000').replace(',',''))
    except : penjualan_list.append("0")

  #menyusun semua list yang sudah didapat menjadi dataset 
  data_tuples = list(zip(nama_list[1:], harga_list[1:], penjualan_list[1:]))
  temp_data = pd.DataFrame(data_tuples, columns=['SKU', 'Harga', 'Jumlah_Penjualan'])
  temp_data['Halaman'] = pg+1
  dataset = dataset.append(temp_data)

driver.close()

In [118]:
df = pd.DataFrame(dataset)

#merubah tipe data harga dan jumlah penjualan menjadi integer
df['Harga'] = df['Harga'].astype('int')
df['Jumlah_Penjualan'] = df['Jumlah_Penjualan'].astype('int')

#Estimasi GMV (harga * jumlah penjualan)
df['GMV'] = df['Harga'] * df['Jumlah_Penjualan']


df.head()

,SKU,Harga,Jumlah_Penjualan,Halaman,GMV
0,ROCKBROS S106 Bike Glove Half Finger – Sarung ...,63900,33000,1,2108700000
1,HELM SEPEDA ROCKBROS TT30 ORIGINAL BICYCLE HEL...,361000,55,1,19855000
2,Lampu Sepeda Led Belakang Rockbros Q1 original...,380000,891,1,338580000
3,ROCKBROS M005BK Topi Sepeda Cycling Cap Bike H...,80750,389,1,31411750
4,ROCKBROS XT-9002 Arm Sleeve - Manset tangan Se...,63960,36000,1,-1992407296


In [119]:
#menyimpan dataset sesuai nama brand dalam format CSV
df.to_csv ("{}.csv".format(brand))